In [19]:
!pip install -U transformers datasets scipy scikit-learn

Requirement already up-to-date: transformers in /usr/local/lib/python3.7/dist-packages (4.8.2)
Requirement already up-to-date: datasets in /usr/local/lib/python3.7/dist-packages (1.8.0)
Requirement already up-to-date: scipy in /usr/local/lib/python3.7/dist-packages (1.7.0)
Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.7/dist-packages (0.24.2)


## 문장 분류 모델 학습

In [20]:
import random
import logging
from IPython.display import display, HTML

import numpy as np
import pandas as pd
import datasets
from datasets import load_dataset, load_metric, ClassLabel, Sequence
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

학습에 필요한 정보를 변수로 기록합니다.

본 노트북에서는 klue-roberta-base 모델을 활용하지만, https://huggingface.co/klue 페이지에서 더 다양한 사전학습 언어 모델을 확인하실 수 있습니다.

학습 태스크로는 nli를, 배치 사이즈로는 32를 지정하겠습니다.

In [21]:
model_checkpoint = "klue/roberta-base"
batch_size = 32
task = "nli"

이제 HuggingFace datasets 라이브러리에 등록된 KLUE 데이터셋 중, NLI 데이터를 내려받습니다.

In [22]:
datasets = load_dataset("klue", task)

Reusing dataset klue (/root/.cache/huggingface/datasets/klue/nli/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90)


In [23]:
datasets

DatasetDict({
    train: Dataset({
        features: ['guid', 'hypothesis', 'label', 'premise', 'source'],
        num_rows: 24998
    })
    validation: Dataset({
        features: ['guid', 'hypothesis', 'label', 'premise', 'source'],
        num_rows: 3000
    })
})

In [24]:
datasets["train"][0]

{'guid': 'klue-nli-v1_train_00000',
 'hypothesis': '힛걸 진심 최고로 멋지다.',
 'label': 0,
 'premise': '힛걸 진심 최고다 그 어떤 히어로보다 멋지다',
 'source': 'NSMC'}

## 시각화 함수

In [25]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."

    picks = []
    
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)

        # 이미 등록된 예제가 뽑힌 경우, 다시 추출
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)

        picks.append(pick)

    # 임의로 추출된 인덱스들로 구성된 데이터 프레임 선언
    df = pd.DataFrame(dataset[picks])

    for column, typ in dataset.features.items():
        # 라벨 클래스를 스트링으로 변환
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])

    display(HTML(df.to_html()))

앞서 정의한 함수를 활용해 훈련 데이터를 살펴보도록 합시다.

이처럼 데이터를 살펴보는 것의 장점으로는 각 라벨에 어떠한 문장들이 해당하는지에 대한 감을 익힐 수 있다는데에 있습니다.

KLUE NLI는 entailment, neutral 그리고 contradiction 세 개의 라벨을 지니는 데이터셋임을 확인할 수 있습니다.

In [26]:
show_random_elements(datasets["train"])

,guid,hypothesis,label,premise,source
0,klue-nli-v1_train_07128,러브 라이브는 학교 아이돌에 관한내용이다.,entailment,"또한, 러브라이브는 오토노키자카 학원을 구하기 위해 코사카 호노카를 비롯한 총 9명의 뮤즈 부원들이 학교 아이돌을 하는 내용이다.",wikinews
1,klue-nli-v1_train_08462,허쉬의 카카오 농장이 전세계에서 가장 크다.,neutral,미국 최대의 초콜릿 제조업체인 허쉬 역시 코트 디부아르 뿐 아니라 서 아프리카 지역에 몇 개의 카카오 농장을 운영하고 있습니다.,wikinews
2,klue-nli-v1_train_07972,버스가 가장 편리하였습니다.,neutral,"메트로, 버스 등 교통이 매우 편리하였습니다.",airbnb
3,klue-nli-v1_train_24858,와인 맛은 별로였습니다.,contradiction,환영하는 의미로 준 와인 맛도 좋았습니다.,airbnb
4,klue-nli-v1_train_00380,2편의 흥행을 재현하려 했으나 1편을 재현한 졸작.,contradiction,1편의 흥행을 재현하려 했으나 2편을 재현한 졸작,NSMC
5,klue-nli-v1_train_10771,센터에 세워지는 건물들은 금속가공 산업의 육성책 중 하나이다.,neutral,센터에는 금속가공 중소기업들이 이용할 수 있는 3개동의 건물과 특수열처리 등 12종의 대형장비가 구축된다.,wikitree
6,klue-nli-v1_train_02807,곤지암 정신병원에는 귀신이 나온다는 괴담이 돌았다.,entailment,귀신이 나온다는 괴담이 돌며 실제 체험을 위해 방문했던 사람들이 실종되었다는 곤지암 정신병원에 남녀 7인으로 이루어진 체험단이 방문한다.,wikipedia
7,klue-nli-v1_train_11476,처음부터 끝까지 지루한 영화였다.,contradiction,스케일부터가 남다름 처음부터 끝까지 지루함이란 찾아볼수없었다,NSMC
8,klue-nli-v1_train_16718,이번 결정에 국방위원회의 명의는 없었다.,contradiction,"이번 결정은 조선로동당 중앙위원회를 비롯한 당 중앙군사위원회, 국방위원회, 최고인민회의 상임위원회 명의로 나온 것으로 알려졌다.",wikinews
9,klue-nli-v1_train_08369,문화재위원회 지정 심의 시에는 다양한 분야의 전문가가 모여 검토를 진행한다.,neutral,문화재청은 지정 예고 기간 동안 제출된 의견은 문화재위원회 지정 심의 시 검토할 예정이라고 밝혔다.,wikinews


훈련 과정 중 모델의 성능을 파악하기 위한 메트릭을 설정합니다.

datasets 라이브러리에는 이미 구현된 메트릭을 사용할 수 있는 load_metric 함수가 있습니다.

그 중 GLUE 데이터셋에 이미 다양한 메트릭이 구현되어 있으므로, GLUE 그 중에서도 KLUE NLI와 동일한 accuracy 메트릭을 사용하는 qnli 태스크의 메트릭을 사용합니다.

In [27]:
metric = load_metric("glue", "qnli")

accuracy 메트릭이 정상적으로 작동하는지 확인하기 위해, 랜덤한 예측 값과 라벨 값을 생성합니다.

In [28]:
fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
fake_preds, fake_labels

(array([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1,
        1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1]),
 array([1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
        0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1]))

In [29]:

metric.compute(predictions=fake_preds, references=fake_labels)

{'accuracy': 0.46875}

## 학습에 활용할 토크나이저를 로드

In [30]:

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

로드된 토크나이저가 두 개 문장을 토큰화하는 방식을 파악하기 위해 두 문장을 입력 값으로 넣어줘보도록 합시다.

In [31]:
tokenizer("힛걸 진심 최고로 멋지다.", "힛걸 진심 최고다 그 어떤 히어로보다 멋지다")

{'input_ids': [0, 3, 7254, 3841, 2200, 11980, 2062, 18, 2, 3, 7254, 3841, 2062, 636, 3711, 12717, 2178, 2062, 11980, 2062, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

input_ids를 보시면 cls_token에 해당하는 2번 토큰이 가장 좌측에 붙게 되며, sep_token의 3번 토큰이 각각 중간과 가장 우측에 더해진 것을 확인할 수 있습니다.

이제 앞서 로드한 데이터셋에서 각 문장에 해당하는 value 를 뽑아주기 위한 key 를 정의합니다.

앞서 KLUE NLI 데이터셋의 두 문장은 각각 premise와 hypothesis라는 이름으로 정의된 것을 확인하였으니, 두 문장의 key 는 마찬가지로 각각 premise, hypothesis가 되게 됩니다.

In [32]:
sentence1_key, sentence2_key = ("premise", "hypothesis")
print(f"Sentence 1: {datasets['train'][0][sentence1_key]}")
print(f"Sentence 2: {datasets['train'][0][sentence2_key]}")

Sentence 1: 힛걸 진심 최고다 그 어떤 히어로보다 멋지다
Sentence 2: 힛걸 진심 최고로 멋지다.


이제 key 도 확인이 되었으니, 데이터셋에서 각 예제들을 뽑아와 토큰화 할 수 있는 함수를 아래와 같이 정의해줍니다.

해당 함수는 모델을 훈련하기 앞서 데이터셋을 미리 토큰화 시켜놓는 작업을 위한 콜백 함수로 사용되게 됩니다.

인자로 넣어주는 truncation는 모델이 입력 받을 수 있는 최대 길이 이상의 토큰 시퀀스가 들어오게 될 경우, 최대 길이 기준으로 시퀀스를 자르라는 의미를 지닙니다.

( * return_token_type_ids는 토크나이저가 token_type_ids를 반환하도록 할 것인지를 결정하는 인자입니다. transformers==4.7.0 기준으로 token_type_ids가 기본적으로 반환되므로 token_type_ids 자체를 사용하지 않는 RoBERTa 모델을 활용하기 위해 해당 인자를 False로 설정해주도록 합니다.)

In [33]:
def preprocess_function(examples):
    return tokenizer(
        examples[sentence1_key],
        examples[sentence2_key],
        truncation=True,
        return_token_type_ids=False,
    )

앞서 정의한 process_function은 여러 개의 예제 데이터를 받을 수도 있습니다.

In [34]:
preprocess_function(datasets["train"][:5])

{'input_ids': [[0, 3, 7254, 3841, 2062, 636, 3711, 12717, 2178, 2062, 11980, 2062, 2, 3, 7254, 3841, 2200, 11980, 2062, 18, 2], [0, 3911, 2377, 2366, 1521, 3061, 4785, 1282, 2955, 3308, 3515, 2170, 22, 2532, 5675, 2, 3911, 2377, 2366, 1525, 2062, 18, 2], [0, 3911, 2377, 2366, 1521, 3061, 4785, 1282, 2955, 3308, 3515, 2170, 22, 2532, 5675, 2, 1282, 2955, 3308, 2052, 3944, 11580, 2359, 2062, 18, 2], [0, 3911, 2377, 2366, 1521, 3061, 4785, 1282, 2955, 3308, 3515, 2170, 22, 2532, 5675, 2, 3911, 2377, 2366, 5105, 2318, 831, 717, 2886, 2069, 575, 555, 2062, 18, 2], [0, 10522, 2548, 2500, 6328, 2170, 6189, 5916, 4015, 2116, 1039, 2219, 3606, 18, 2, 10522, 2548, 2500, 6328, 27135, 5916, 4015, 1642, 2015, 2259, 4258, 2219, 3606, 18, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1

이제 정의된 전처리 함수를 활용해 데이터셋을 미리 토큰화시키는 작업을 수행합니다.

datasets 라이브러리를 통해 얻어진 DatasetDict 객체는 map() 함수를 지원하므로, 정의된 전처리 함수를 데이터셋 토큰화를 위한 콜백 함수로 map() 함수 인자로 넘겨주면 됩니다.

보다 자세한 내용은 문서를 참조해주시면 됩니다.

In [35]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

## 학습을 위한 모델을 로드

앞서 살펴본 바와 같이 KLUE NLI에는 총 3개의 클래스가 존재하므로, 3개의 클래스를 예측할 수 있는 SequenceClassification 구조로 모델을 로드하도록 합니다.

In [36]:
num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifie

모델을 로드할 때 발생하는 경고 문구는 두 가지 의미를 지닙니다.

Masked Language Modeling 을 위해 존재했던 lm_head가 현재는 사용되지 않고 있음을 의미합니다.
문장 분류를 위한 classifier 레이어를 백본 모델 뒤에 이어 붙였으나 아직 훈련이 되지 않았으므로, 학습을 수행해야 함을 의미합니다.
마지막으로 앞서 정의한 메트릭을 모델 예측 결과에 적용하기 위한 함수를 정의합니다.

입력으로 들어오는 eval_pred는 EvalPrediction 객체이며, 모델의 클래스 별 예측 값과 정답 값을 지닙니다.

클래스 별 예측 중 가장 높은 라벨을 argmax()를 통해 뽑아낸 후, 정답 라벨과 비교를 하게 됩니다.

In [37]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)


이제 앞서 정의한 정보들을 바탕으로 transformers에서 제공하는 Trainer 객체를 활용하기 위한 인자 관리 클래스를 초기화합니다.

metric_name은 앞서 얻어진 메트릭 함수를 활용했을 때, 아래와 같이 dict 형식으로 결과 값이 반환되는데 여기서 우리가 사용할 key 를 정의해준다고 생각하시면 됩니다.

>>> metric.compute(predictions=fake_preds, references=fake_labels)
{'accuracy': 0.515625}
각 인자에 대한 자세한 설명은 문서에서 참조해주시면 됩니다.

In [38]:
metric_name = "accuracy"

args = TrainingArguments(
    "test-nli",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

이제 로드한 모델, 인자 관리 클래스, 데이터셋 등을 Trainer 클래스를 초기화에 넘겨주도록 합니다.

(TIP: Q: 이미 encoded_datasets을 만드는 과정에 토큰화가 이루어졌는데 토크나이저를 굳이 넘겨주는 이유가 무엇인가요?,
A: 토큰화는 이루어졌지만 학습 과정 시, 데이터를 배치 단위로 넘겨주는 과정에서 배치에 포함된 가장 긴 시퀀스 기준으로 truncation을 수행하고 최대 길이 시퀀스 보다 짧은 시퀀스들은 그 길이만큼 padding을 수행해주기 위함입니다.)

In [39]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

이제 정의된 Trainer 객체를 다음과 같이 훈련시킬 수 있습니다.

에폭이 지남에 따라 Loss 는 떨어지고, 앞서 선정한 메트릭인 Accuracy 는 증가하는 것을 확인할 수 있습니다.

In [40]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: guid, hypothesis, premise, source.
***** Running training *****
  Num examples = 24998
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3910


Epoch,Training Loss,Validation Loss,Accuracy
1,0.607200,0.432421,0.839333
2,0.279100,0.423450,0.857000
3,0.173900,0.464753,0.858000


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: guid, hypothesis, premise, source.
***** Running Evaluation *****
  Num examples = 3000
  Batch size = 32
Saving model checkpoint to test-nli/checkpoint-782
Configuration saved in test-nli/checkpoint-782/config.json
Model weights saved in test-nli/checkpoint-782/pytorch_model.bin
tokenizer config file saved in test-nli/checkpoint-782/tokenizer_config.json
Special tokens file saved in test-nli/checkpoint-782/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: guid, hypothesis, premise, source.
***** Running Evaluation *****
  Num examples = 3000
  Batch size = 32
Saving model checkpoint to test-nli/checkpoint-1564
Configuration saved in test-nli/checkpoint-1564/config.json
Model weights saved in test-nli/checkpoint-1564

Epoch,Training Loss,Validation Loss,Accuracy
1,0.607200,0.432421,0.839333
2,0.279100,0.423450,0.857000
3,0.173900,0.464753,0.858000
4,0.097100,0.604402,0.865667
5,0.070100,0.705257,0.864333


Saving model checkpoint to test-nli/checkpoint-3128
Configuration saved in test-nli/checkpoint-3128/config.json
Model weights saved in test-nli/checkpoint-3128/pytorch_model.bin
tokenizer config file saved in test-nli/checkpoint-3128/tokenizer_config.json
Special tokens file saved in test-nli/checkpoint-3128/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: guid, hypothesis, premise, source.
***** Running Evaluation *****
  Num examples = 3000
  Batch size = 32
Saving model checkpoint to test-nli/checkpoint-3910
Configuration saved in test-nli/checkpoint-3910/config.json
Model weights saved in test-nli/checkpoint-3910/pytorch_model.bin
tokenizer config file saved in test-nli/checkpoint-3910/tokenizer_config.json
Special tokens file saved in test-nli/checkpoint-3910/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/mo

TrainOutput(global_step=3910, training_loss=0.22737782080764965, metrics={'train_runtime': 1794.0685, 'train_samples_per_second': 69.668, 'train_steps_per_second': 2.179, 'total_flos': 5701283955554040.0, 'train_loss': 0.22737782080764965, 'epoch': 5.0})

In [41]:
TrainOutput(global_step=3910, training_loss=0.21999031222994675, metrics={'train_runtime': 1761.9721, 'train_samples_per_second': 70.938, 'train_steps_per_second': 2.219, 'total_flos': 5701283955554040.0, 'train_loss': 0.21999031222994675, 'epoch': 5.0})

NameError: ignored

Trainer 는 학습을 마치게 되면, load_best_model_at_end=True 인자에 따라 메트릭 기준 가장 좋은 성능을 보였던 체크포인트를 로드하게 됩니다.

본 노트북에서는 마지막 에폭 때 가장 좋은 성능을 얻었기에 evaluate를 수행해도 같은 결과가 나오겠습니다.

In [ ]:
trainer.evaluate()

지금까지 transformers를 라이브러리 내 문장 분류 모델을 학습하는 과정을 KLUE NLI 데이터셋을 통해 알아보았습니다.

본 노트북을 통해 습득한 지식이 여러분의 업무와 학습에 도움이 되었으면 좋겠습니다.

허 훈 (huffonism@gmail.com)
APPENDIX: 앞서 학습된 모델을 HuggingFace 모델 허브에 업로드하였으니, 아래 예제와 같이 pipeline 함수를 통해 사용이 가능합니다.

먼저 text-classification 태스크로 파이프라인 객체를 초기화합니다.

( * return_all_scores는 모델이 입력 문장에 대해 측정한 각 라벨에 대한 확률 값을 모두 보여줄 것인지를 결정하는 인자입니다.)

In [ ]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="Huffon/klue-roberta-base-nli",
    return_all_scores=True,
)

NLI는 두 문장의 페어를 입력 값으로 주어야 하기 때문에 구분자 스페셜 토큰을 두 문장 사이에 넣어주기 위해 토크나이저 객체를 로드합니다.

[SEP] 문자열을 하드코딩하여 넣어줄 수도 있겠지만, 스페셜 토큰은 토크나이저 마다 다르게 정의되므로 다른 모델을 활용할 때 보다 코드를 재사용할 수 있도록 토크나이저의 sep_token 프로퍼티에 접근하는 방식으로 코드를 작성합니다.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Huffon/klue-roberta-base-nli")

In [ ]:
tokenizer.sep_token

sep_token으로 두 문장을 이어 하나의 입력 값으로 파이프라인에 넘겨줍니다.

입력된 문장에 대해 모델이 각 라벨에 대해 어떤 확률을 가지고 예측했는지를 확인할 수 있습니다.

In [ ]:
classifier(f"흡연하려면 발코니 있는 방을 선택하면 됩니다. {tokenizer.sep_token} 흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다.")

In [ ]:
classifier(f"호스트분은 영어밖에 못하십니다. {tokenizer.sep_token} 호스트분도 엄청 친절하시고 영어도 잘하십니다.")

검증 데이터에 존재하는 임의의 문장 페어를 입력해보니, 우리가 원하던 결과를 얻을 수 있었습니다.

(cf. NLI 데이터에 대해 학습된 모델을 활용해 Zero-shot Classification을 수행하는 예제가 궁금하신 분들은 해당 노트북을 참조해주세요.)